In [1]:
# Add to environment variables PYTHONPATH = C:\Users\JFC_WIN\Documents\OpenSimula\src
import OpenSimula as osm
import pandas as pd

bestest_building_600 = {
    "name": "Bestest Building",
    "time_step": 3600,
    "n_time_steps": 8760,
    "initial_time": "01/01/2001 00:00:00",
    "components": [
        # MET_FILE
        {
            "type": "File_met",
            "name": "sevilla",
            "file_name": "met_files/sevilla.met"
        },
        # CONSTRUCTION
        {
            "type": "Material",
            "name": "Enlucido",
            "conductivity": 0.16,
            "density": 950,
            "specific_heat": 840
        },
        {
            "type": "Material",
            "name": "Aislamiento",
            "conductivity": 0.04,
            "density": 12,
            "specific_heat": 840
        },
        {
            "type": "Material",
            "name": "Madera pared",
            "conductivity": 0.14,
            "density": 530,
            "specific_heat": 900
        },
        {
            "type": "Material",
            "name": "Aislamiento suelo",
            "conductivity": 0.04,
            "density": .1,
            "specific_heat": .1,
        },
        {
            "type": "Material",
            "name": "Madera suelo",
            "conductivity": 0.14,
            "density": 650,
            "specific_heat": 1200,
        },
        {
            "type": "Material",
            "name": "Lecho_rocas",
            "conductivity": 0.14,
            "density": 650,
            "specific_heat": 1200,
        },
        {
            "type": "Construction",
            "name": "Pared 600",
            "solar_absortivity": [0.6, 0.6],
            "materials": ["Enlucido", "Aislamiento", "Madera pared"],
            "thicknesses": [0.12, 0.066, 0.009],
        },
        {
            "type": "Construction",
            "name": "Suelo 600",
            "solar_absortivity": [0.6, 0.6],
            "materials": ["Aislamiento suelo", "Madera suelo"],
            "thicknesses": [1.003, 0.025],
        },
        {
            "type": "Construction",
            "name": "Techo 600",
            "solar_absortivity": [0.6, 0.6],
            "materials": ["Enlucido", "Aislamiento", "Madera pared"],
            "thicknesses": [0.010, 0.01118, 0.019],
        },
        {
            "type": "Window",
            "name": "Vidrio doble",
            "solar_absortivity": [0.13, 0.13],
            "solar_transmisivity": [0.66, 0.66],
            "R_glazing": 0.921,
            "R_frame": 0.921,
            "frame_fraction": 0
        },
        # BUILDING
        {
            "type": "Building",
            "name": "Bestest 600",
            "file_met": "sevilla"
        },
        {
            "type": "Space_type",
            "name": "bestest space",
            "aux_variables": [],
            "people_density": "0",
            "light_density": "0",
            "other_gains_density": "4.1667",
            "other_gains_radiant_fraction": 0.6,
            "infiltration": "0.5"
        },
        {
            "type": "Space",
            "name": "space_1",
            "building": "Bestest 600",
            "space_type": "bestest space",
            "floor_area": 48,
            "volume": 48*2.7,
        },
        # Surfaces
        {
            "type": "Exterior_surface",
            "name": "north_wall",
            "virtual": False,
            "construction": "Pared 600",
            "space": "space_1",
            "area": 8*2.7,
            "azimuth": 180,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "east_wall",
            "virtual": True,
            "space": "space_1",
            "area": 6*2.7,
            "azimuth": 90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "south_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "area": 8*2.7,
            "azimuth": 0,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "west_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "area": 6*2.7,
            "azimuth": -90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "roof",
            "construction": "Techo 600",
            "space": "space_1",
            "area": 8*6,
            "azimuth": 0,
            "altitude": 90
        },
        {
            "type": "Underground_surface",
            "name": "floor",
            "construction": "Suelo 600",
            "space": "space_1",
            "area": 8*6,
            "azimuth": 0,
            "altitude": -90
        },
    ],
}

In [2]:
sim = osm.Simulation()
pro = osm.Project("pro", sim)
pro.read_dict(bestest_building_600)
pro.simulate()

Reading project data from dictonary
Reading completed.
Checking project: Bestest Building
Simulating Bestest Building: 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  End


In [3]:
pro.component("Bestest 600").SWDIF_matrix

array([[ 0.        , -2.08734347,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        , -2.08734347,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        , -1.55347981,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        , -4.70927512,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        , -4.70927512,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [9]:
wall = "south_wall"
df_met = pro.component("sevilla").variable_dataframe()
df_space = pro.component("space_1").variable_dataframe()
df_wall = pro.component(wall).variable_dataframe()

df_graph = pd.DataFrame()
df_graph["date"] = df_met["date"]
df_graph["T_ext"] = df_met["temperature [°C]"]
df_graph["T_z"] = df_space["temperature [°C]"]
df_graph["T_s0"] = df_wall["T_s0 [°C]"]
df_graph["T_s1"] = df_wall["T_s1 [°C]"]

df =pro.component("sevilla").variable_dataframe(frequency="m",value="max")
df
#pro.component(wall).area



,sol_hour [h],temperature [°C],sky_temperature [°C],underground_temperature [°C],rel_humidity [%],sol_direct [W/m²],sol_diffuse [W/m²],wind_speed [m/s],wind_direction [°],sol_azimuth [°],sol_altitude [°]
date,,,,,,,,,,,
2001-01-31,21.553382,19.788503,8.769164,17.604463,99.822039,465.202640,254.309375,13.987889,358.426486,63.302100,34.567229
2001-02-28,21.383772,21.948251,10.289507,17.604463,93.599848,552.982566,317.561436,12.519487,359.225103,70.217404,43.632716
2001-03-31,21.486337,24.988258,13.197574,17.604463,85.809296,835.042974,339.683521,14.559576,357.983488,88.109576,55.605549
2001-04-30,20.649750,27.476144,16.170112,17.604463,84.608607,960.560744,332.804321,15.647746,358.866455,104.761084,66.731076
2001-05-31,20.667413,32.100000,24.030974,17.604463,93.898412,1006.316127,353.762942,14.298905,357.488812,110.657236,73.854745
2001-06-30,20.644727,34.391663,27.465193,17.604463,83.425820,1049.206675,360.873739,14.714830,358.265382,112.571296,75.071401
2001-07-31,20.544086,38.500000,26.892097,17.604463,73.959060,940.000000,418.532305,14.584669,358.948166,112.514194,74.574817
2001-08-31,20.590002,37.093136,25.255387,17.604463,67.841087,861.696577,366.437910,14.382738,357.497303,108.785657,69.475592
2001-09-30,20.770661,34.817336,25.457365,17.604463,74.182930,788.576859,331.024694,14.205891,359.728201,90.784164,59.834827


In [5]:
import plotly.express as px

fig = px.line(df_graph,x='date',y=['T_ext','T_z','T_s0','T_s1'],height=400)
fig.show()